## Steps
- 1. Get the live video feed from your webcam
- 2. Fix a scene (the place you want to monitor) and store it as a reference background image
    - Store the first frame as the reference background frame
- 3. For every frame, check if there is any unwanted object inside the scene you are monitoring
    - Use **Background Subtraction** concept (**cv2.absdiff( )**)
        - Subtract the current frame from the reference background image(frame) to see the changes in the scene
        - If there is enormous amount of pixels distrubed in the subtraction result image
            - unwanted visitor (place is unsafe --> alarm the authorities)
        - If there is no enormous amount of pixels distrubed in the subtraction result image
            - no unwanted visitor (place is safe)
- 4. Output the text **"UNSAFE"** in **red** color on the top right of the frame when there is an intruder in the scene.
- 5. Save the live feed
- 6. Submit the (.ipynb) file

- For devices using wayland, opencv may crash the notebook if using cv2.imshow(), thats why this notebook use matplotlib to display the images
- For devices not using wayland replace `plt.imshow(...)` to `cv2.imshow("...", ...)` and replace `plt.show()` to `cv2.waitKey(0)`

## Get live video feed from webcam [10 points]

In [ ]:
# !pip3 install opencv-python matplotlib numpy
import cv2
import numpy as np
from matplotlib import pyplot as plt
video_capture = cv2.VideoCapture(0)


## Read first frame, convert to Grayscale and store it as reference background image

In [ ]:
ret, frame = video_capture.read()
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
plt.imshow(gray, cmap = "gray")
plt.show()

## Compute Absolute Difference between Current and First frame

In [ ]:
save = cv2.VideoWriter('output_0.avi', cv2.VideoWriter_fourcc(*"mp4v"), 10, (1280,720))
while True:
    ret, frame = video_capture.read()
    save.write(frame)
    if not ret:
        print("Error: Could not read frame.")
        break
    current_frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    abs_diff = cv2.absdiff(gray, current_frame_gray)
    cv2.imshow('Absolute Difference', abs_diff)
# press esc
    if cv2.waitKey(30) & 0xFF == 27:
        cv2.destroyAllWindows()
        break
save.release()
plt.imshow(frame, cmap="gray")
plt.show()

In [ ]:
plt.imshow(abs_diff, cmap="gray")
plt.show()


## Apply threshold

In [ ]:
thresholded = cv2.GaussianBlur(abs_diff, (3,3),0)
thresholded = cv2.erode(thresholded, None, iterations=3)
thresholded = cv2.dilate(thresholded, None, iterations=3)

thresholded = cv2.threshold(thresholded, 20, 255, cv2.THRESH_BINARY_INV)[1]
plt.imshow(thresholded, cmap="gray")
plt.show()


## Find contours

In [ ]:
# thresholded = cv2.Canny(thresholded,20,185)
# plt.imshow(thresholded,cmap="gray")

In [ ]:
contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
frame_with_contours = thresholded.copy()
cv2.drawContours(frame_with_contours, contours, -1, (0, 255, 0), 2)  # -1 means draw all contours
plt.imshow(frame_with_contours,cmap="gray")
plt.show()


## Check if contourArea is large and draw rectangle around the object, output "UNSAFE" text in red color

In [ ]:
  for contour in contours:
        area = cv2.contourArea(contour)
        if area > 1000: 
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2) 
            cv2.putText(frame, "UNSAFE", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)


## Display images

In [ ]:
plt.imshow(frame)
plt.show()

## Release objects

In [ ]:
video_capture.release()
